In [1]:
import requests 
import json
import pandas as pd
import warnings
import pyecharts as pe

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "/Users/chaozy/anaconda3/lib/python3.6/site-packages/lml/utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [3]:
# GET Access Token
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=E52brsYEkEv9bG7xwqecY64S&client_secret=BAaNBmr3gV7PHIfMFbYBd3QVfT4sFQC9'
response = requests.get(host)
if response:
    access_token = response.json()['access_token']
# Fill in header parameters
headers = {'content-type': 'application/json'}

# GENERATE url
url_sentiment_classify = 'https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?charset=UTF-8&access_token=' +  access_token

with open("SunYung.txt", 'r') as file:
    data = file.read().replace('\n', '')
data

'孙杨的事情，还没有结束。最近，他的妈妈杨明，也被卷进了舆论风波中。在国际体育仲裁法庭宣布孙杨禁赛八年后，爱子心切的孙妈妈，在朋友圈发了一条长文。文章开头，就在哭诉儿子的不容易。之后，她就开始指责领导的不负责，指派律师的不专业。首先，她表明孙杨在08年因为身体不适服用了含有盐酸曲美他嗪的“万爽力。”当时，这药物还未变成“赛内禁用，赛外可用”的违禁药。直到过了6年，中国反兴奋剂网站才下发通知，表明“万爽力”是违禁药，强调运动员对此是毫不知情。当年不知情的孙杨，还是正常使用了“万爽力”，没想到体检是阳性，为此还被口头警告，要求罚款5000元。对于自己被“冤枉”，孙杨当然是不乐意的。但是在领导说出“不会影响成绩”后，孙杨妈妈选择先把这件事情放下，让儿子继续比赛。孙杨，也这么做了。听话的后果，就让这件事成为了“孙杨职业生涯最大的耻辱柱。”再之后参加国际听证会和仲裁会，孙杨妈妈也认为领导指派的律师专业性十分不强，影响了最终结果的判断，导致了自己儿子被禁赛8年的事情发生。整篇文章看下来，可以看出孙杨妈妈的不甘和怒气。也能看出，她意在把事情的全部责任，都推卸到别人身上。但是，真相似乎并不像孙杨妈妈所指的一样，孙杨被禁赛长达8年，只有别人的责任，自己毫无问题。前不久，有人在网上放出了孙杨妈妈在听证会的一段视频曝光。在听证会上，孙杨妈妈一直不愿正面回答对方提出的问题。并且不顾规则，总是中途打断对方，浪费了许多时间。法官问了三遍“孙杨是否撕毁了他之前签字的检验单？”孙杨妈妈是承认了，但却一直解释，自己儿子的暴力抗检，是“有理有据。”明明是回答是或者不是的问题，孙杨妈妈却一直自说自话，胡乱辩解。直到她的证词时间已经结束了，她还要强调“我还没说完。”不听安排，无视规则。而且据爆料，孙杨妈妈在去年6月，还曾私下联系过血检官和尿检官。这些，都是WADA明令禁止的。这一系列操作看下来，孙杨妈妈对于整件事而言，没有任何正面帮助，还帮了倒忙。在孙杨的成长道路，运动员生涯中，最离不开和最常见到的身影，就是他的妈妈杨明。作为曾经的浙江女排主力，专业的运动员，杨明深知运动员锻炼时间的重要性。在孙杨小时候，杨明是风雨无阻开车送儿子去训练，完全没有自我时间。为了让孩子把一切时间运用在游泳上，杨明更是把孙杨当成了“巨婴”去养。小学六年级，孙杨还不会系鞋带，上学训练也是从不背书包。在杨明的眼里，这些都是小事，什么

In [5]:
# GET sentences that contain Sun Yung

sy_sentences = []
for i in data.split('。'):
    if '孙杨' in i:
        sy_sentences.append(i)
sy_sentences[:5]

包含孙杨的句子数量： 66


['孙杨的事情，还没有结束',
 '在国际体育仲裁法庭宣布孙杨禁赛八年后，爱子心切的孙妈妈，在朋友圈发了一条长文',
 '首先，她表明孙杨在08年因为身体不适服用了含有盐酸曲美他嗪的“万爽力',
 '当年不知情的孙杨，还是正常使用了“万爽力”，没想到体检是阳性，为此还被口头警告，要求罚款5000元',
 '对于自己被“冤枉”，孙杨当然是不乐意的']

In [52]:
def sentiment_ana(data):
    # perform word analysis on this article
    response = requests.post(url_sentiment_classify, 
        data = json.dumps({'text': data}),  
        headers = headers)
    if response:
        result = response.json()['items'][0]
        result_pos = result["positive_prob"]
        result_sentiment = result["sentiment"]
        return result_pos, result_sentiment


In [53]:
def fullText_sentiment_ana(article):
    result  = []
    index = 0
    for sentence in article:
        try:
            positive, senti = sentiment_ana(sentence)
            result.append({"sentence" : sentence, "Positive_prob" : positive, "Sentiment" : senti})
        except:
            continue
    return result
lis = fullText_sentiment_ana(sy_sentences)


In [59]:
ana_table = pd.DataFrame(lis)
ana_table
len(lis)

14

In [55]:
# DRAW the bar chart and line diagram to present data
# define x and y axises
x = ana_table['sentence']
y1 = ana_table['Positive_prob']
y2 = ana_table['Sentiment']

bar = pe.Bar("Sun Yung Postivity")
bar.add("Emotion_positivity", x, y1, is_datazoom_show=True, 
        datazoom_range=[0,100], mark_line=["average"], tooltip_axispointer_type="cross")

line = pe.Line("Sun Yung Sentiment")
line.add("Emotion_sentiment", x, y2,is_datazoom_show=True,  is_step=True, area_opacity=True)
line

In [56]:
# overlap two diagrams
overLap = pe.Overlap()
overLap.add(line)
overLap.add(bar)
overLap